In [3]:
import pandas as pd
import numpy as np
import re  # string mani
import os  # read fold info
import csv

#### Notes:
* wave use: ADNI1, ADNIGO, ADNI2

* find values start from MCI -> check 6m, 12m, 24m follow up

* make the MCI with A/T/N values as a specific subgroup -> compare the prediction & feature importance

Methods applied: catboost, light gbm, xgboost, gbm, random forest, logistic regression

evaluation: MCC, accuracy...

In [4]:
pd.set_option('display.max_rows', 500)  # let pd.df display() 500 rows
pd.set_option('display.max_columns', None)
# functions; temp
def d_overview(df):
    print(f"Dimension: {df.shape}")
    output = df.apply(lambda x: (x.dtype,x.isna().mean(),x.unique()), axis=0).T
    output.columns = ["dtype", "NaN_precentage","unique"]
    return output.sort_values(by="NaN_precentage", ascending=False)
    
def keep_median_row(group):
    if 'MEDIAN' in group['BATCH'].values:
        return group[group['BATCH'] == 'MEDIAN']
    else:
        return group.head(1)
def basic_clean(df, printna=False):
    df.replace([-1,-4,"-1","-4","Unknown"], np.nan, inplace=True)  # -1&-4 represents missingness in ADNI
    if 'update_stamp' in df.columns:
        df.drop('update_stamp', axis=1, inplace=True)
    df = df.loc[:, (df.isna().mean() <= 0.5) &           # rm >50% missingness
                ~(df.columns.str.contains('DATE'))       # drop date related feature
               ].reset_index(drop=True) 
    df['RID'] = df['RID'].astype(int)
    return df

In [5]:
biomarker_r = pd.read_csv("../rawdata/UPENNBIOMK_MASTER.csv", error_bad_lines=False, engine="python")  # CSF markers info
# > unable to read line 5877 -> checked, no impact(median point of a single batch subject)
symptoms_r = pd.read_csv("../rawdata/ADSXLIST.csv")     # diagnosed symptoms 
adnimerge_r = pd.read_csv("../../ADNIMERGE.csv")     # combined data from ADNI


/var/folders/bm/s0j8htzs7tn4d1t__749xglm0000gn/T/ipykernel_10103/3875110897.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  biomarker_r = pd.read_csv("../rawdata/UPENNBIOMK_MASTER.csv", error_bad_lines=False, engine="python")  # CSF markers info
Skipping line 5877: unexpected end of data
/var/folders/bm/s0j8htzs7tn4d1t__749xglm0000gn/T/ipykernel_10103/3875110897.py:4: DtypeWarning: Columns (19,20,21,104,105,106) have mixed types. Specify dtype option on import or set low_memory=False.
  adnimerge_r = pd.read_csv("../../ADNIMERGE.csv")     # combined data from ADNI


## data cleaning

* combined data set: df_merge

* primary & foreign key for all 3 dataset: RID & VISCODE

#### UPENNBIOMK_MASTER

- note: 
    - feature info: UPENNBIOMK_MASTER_DICT.csv file
    - contain ADNI1, GO, 2
- cleaning:
    - use median of UPENNBIOMK_MASTER from multiple batches (already standardized to ADNI1)
    - labeled CSF cols with **_UPENN**

In [10]:
# Demo of conflicted values in ADNI merge & UPENN.
# Use UPENN since it has more completed code book
print("From ADNI merge:")
print(adnimerge_r.loc[adnimerge_r["RID"]==5,["RID","VISCODE","PTAU"]])
print("From UPENN CSF report:")
print(biomarker_r.loc[biomarker_r["RID"]==5, ["RID","VISCODE","PTAU","BATCH"]])

From ADNI merge:
      RID VISCODE   PTAU
10      5      bl  33.43
11      5     m06    NaN
12      5     m12  34.04
13      5     m24    NaN
14      5     m36    NaN
5116    5     m18    NaN
5117    5     m30    NaN
From UPENN CSF report:
    RID VISCODE  PTAU        BATCH
10    5      bl  68.0   UPENNBIOMK
11    5      bl  66.3  UPENNBIOMK2
12    5      bl  67.2       MEDIAN
13    5     m12  40.5  UPENNBIOMK2
14    5     m12  40.5       MEDIAN


In [11]:
# biomarker.apply(lambda x:sum(x.isin(["-1","-4",-1,-4])))  # no -1,-4 in biomarker
# group by visit -> keep median for multiple batches:
biomarker = biomarker_r.groupby(['RID', 'VISCODE'], as_index=False)\
            .apply(keep_median_row).reset_index(drop=True) 
biomarker = biomarker.loc[:,['RID','VISCODE','ABETA','TAU','PTAU']] # keep useful col
biomarker = biomarker.rename(columns={'ABETA':'ABETA_UPENN','TAU':'TAU_UPENN', 'PTAU':'PTAU_UPENN'})

In [12]:
print(biomarker["VISCODE"].value_counts()) # 1238 bl -> ~3,400 to m12/m24
biomarker[biomarker.duplicated(subset=["RID","VISCODE"])] # no duplicate
d_overview(biomarker_r) # low missingness
# biomarker["RID"].tail() 

bl     1238
m24     402
m12     326
m36      81
m48      67
m60      36
m72      21
m84       3
Name: VISCODE, dtype: int64
Dimension: (5875, 14)


,dtype,NaN_precentage,unique
TAU_RAW,float64,0.381957,"[68.0, 55.5, nan, 77.5, 42.0, 41.9, 49.9, 112...."
ABETA_RAW,float64,0.372936,"[131.0, 149.0, nan, 155.0, 256.0, 317.0, 303.0..."
PTAU_RAW,float64,0.371574,"[21.0, 12.8, nan, 13.7, 13.0, 8.66, 9.04, 68.0..."
TAU,float64,0.014979,"[68.0, 54.9, 61.4, 76.5, 42.0, 41.5, 41.8, 49...."
PTAU,float64,0.004936,"[21.0, 19.8, 20.4, 21.1, 13.0, 12.9, 13.5, 68...."
ABETA,float64,0.003234,"[131.0, 132.0, 137.0, 256.0, 246.0, 115.0, 117..."
RID,int64,0.0,"[3, 4, 5, 8, 10, 14, 16, 19, 22, 23, 29, 31, 3..."
VISCODE,object,0.0,"[bl, m12, m24, m48, m60, m84, m36, m72]"
BATCH,object,0.0,"[UPENNBIOMK, UPENNBIOMK2, MEDIAN, UPENNBIOMK3,..."
KIT,object,0.0,"[170470, 190841, ALL, D191113, 200413, 225445,..."


#### ADSXLIST

- note:
    
    - feature info: DATADIC.csv 
    
    - BLSCHECK.csv: pre-existing symptoms (decide not to use, recall bias)

    - ADSXLIST: diagnosed symptoms, only have ADNI1 & ADNIGO
    
- cleaning

    - -1, -4 to NaN

    - ori label: 1=Absent; 2=Present -> change to 0=absent, 1=present
    
    - rm col with >50% missingness & VISCODE2

In [13]:
symptoms = basic_clean(symptoms_r) 
symptoms = symptoms.rename(columns=lambda x: x.replace('AX', ''))
symptoms.iloc[:,6:] = symptoms.iloc[:,6:]-1
symptoms[symptoms.duplicated(subset=["RID","VISCODE"])] # no duplicate
sum(symptoms.VISCODE2 != symptoms.VISCODE) # exact same, rm VISCODE2 since other 2 use VISCODE
symptoms.drop(['VISCODE2','SITEID','ID','Phase'], axis=1, inplace=True)

In [14]:
display(symptoms.iloc[:,5:].apply(pd.value_counts, axis=0)) # quite skewed
d_overview(symptoms)

,CONSTP,ABDOMN,SWEATN,DIZZY,ENERGY,DROWSY,VISION,HDACHE,DRYMTH,BREATH,COUGH,PALPIT,CHEST,URNDIS,URNFRQ,ANKLE,MUSCLE,RASH,INSOMN,DPMOOD,CRYING,ELMOOD,WANDER,FALL,OTHER
0.0,4337,4542,4619,4264,3798,4166,4586,4563,4364,4461,4302,4730,4733,4771,3683,4415,3072,4514,4255,4166,4688,4811,4797,4498,3807
1.0,538,332,256,611,1076,709,289,312,511,414,573,145,142,104,1192,460,1803,360,619,707,186,63,77,376,1061


Dimension: (4884, 30)


,dtype,NaN_precentage,unique
OTHER,float64,0.003276,"[0.0, 1.0, nan]"
DPMOOD,float64,0.002252,"[0.0, 1.0, nan]"
FALL,float64,0.002048,"[0.0, 1.0, nan]"
WANDER,float64,0.002048,"[0.0, 1.0, nan]"
ELMOOD,float64,0.002048,"[0.0, 1.0, nan]"
CRYING,float64,0.002048,"[0.0, 1.0, nan]"
ABDOMN,float64,0.002048,"[0.0, 1.0, nan]"
INSOMN,float64,0.002048,"[0.0, 1.0, nan]"
ENERGY,float64,0.002048,"[0.0, 1.0, nan]"
RASH,float64,0.002048,"[0.0, 1.0, nan]"


#### ADNIMERGE

- feature info: ADNIMERGE_DICT.csv

- cleaning:
    - remove 4858 rows with no outcome(DX)
    - no missingness in VISCODE(timepoint)
    - remove all the features with _bl (repeatative); see below chunk as demo
    - remove time & ID-related cols expect RID & VISCODE
    - didn't remove any cols with high missingness

In [15]:
# demo of _bl features -> useless
check_bl = adnimerge_r[["RID","VISCODE","MMSE_bl","MMSE"]].dropna()
check_bl.loc[check_bl["RID"]==3]

,RID,VISCODE,MMSE_bl,MMSE
1,3,bl,20.0,20.0
2,3,m06,20.0,24.0
3,3,m12,20.0,17.0
4,3,m24,20.0,19.0


In [1]:
adnimerge_r.shape

NameError: name 'adnimerge_r' is not defined

In [16]:
# adnimerge_r.loc[adnimerge_r["ORIGPROT"]=="ADNI3"]["VISCODE"].value_counts()
# > ADNI3 still have quite a few time points
adnimerge = adnimerge_r.loc[:,
                  ~(adnimerge_r.columns.str.contains('(DATE|Month|Year)')) & # drop date related feature
                  ~(adnimerge_r.columns.str.contains('_bl$'))
               ].reset_index(drop=True)
adnimerge.drop(['COLPROT',"PTID",'update_stamp','M'], axis=1, inplace=True)
adnimerge = adnimerge.loc[~adnimerge["DX"].isnull()]
d_overview(adnimerge)
# adnimerge.VISCODE[adnimerge.VISCODE.isnull()]  # no missing in VISCODE

Dimension: (11364, 60)


/var/folders/bm/s0j8htzs7tn4d1t__749xglm0000gn/T/ipykernel_52908/2588660186.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ~(adnimerge_r.columns.str.contains('(DATE|Month|Year)')) & # drop date related feature


,dtype,NaN_precentage,unique
PIB,float64,0.980465,"[nan, 2.3575, 2.1275, 2.0725, 2.19, 2.27, 2.35..."
FBB,float64,0.955209,"[nan, 1.0714, 1.0372, 0.9902, 0.9959, 0.9602, ..."
PTAU,object,0.791799,"[nan, 22.83, 24.18, 13.29, 13.3, 33.43, 34.04,..."
ABETA,object,0.791711,"[nan, 741.5, 601.4, 1501.0, 1176.0, 547.3, 472..."
TAU,object,0.791711,"[nan, 239.7, 251.7, 153.1, 159.7, 337.0, 334.1..."
AV45,float64,0.731345,"[nan, 1.3496, 0.9804, 0.9851, 0.994, 0.9416, 0..."
FDG,float64,0.675818,"[1.33615, 1.1086, 1.07796, 1.11626, 1.06134, n..."
DIGITSCOR,float64,0.665611,"[34.0, 25.0, 19.0, 21.0, 16.0, 24.0, 30.0, 28...."
EcogSPOrgan,float64,0.38534,"[nan, 1.0, 1.25, 1.33333, 2.33333, 2.8, 3.5, 1..."
FLDSTRENG,object,0.38446,"[1.5 Tesla MRI, 3 Tesla MRI, nan]"


### Merge data

- left join adnimerge because all keys from biomarker & symptoms exist in adnimerge

- didn't do any imputation

In [24]:
# merge full data:
df_3to1 = pd.merge(
    pd.merge(adnimerge,biomarker, on=['RID', 'VISCODE'], how="left"),
    symptoms, how="left"
)
df_3to1.replace(["Unknown"], np.nan, inplace=True)

### Re-label:

- DX: {'CN': 1, 'MCI': 2, 'Dementia': 3}
- VISCODE: {"bl":0, "m_" :_ } ; eg. m12:12

- below will be labeled as 0, 1, 2...
    
    - ORIGPROT: {ADNI1:0, ADNIGO:1, ADNI2:2, ADNI3:3}
    - PTGENDER: {'Male', 'Female'}
    - PTETHCAT: {'Not Hisp/Latino', 'Hisp/Latino'}
    - PTRACCAT: {'White', 'More than one', 'Black', 'Asian', 'Am Indian/Alaskan', 'Hawaiian/Other PI'}
    - PTMARRY: {'Married', 'Divorced', 'Widowed', 'Never married'}
    - FLDSTRENG: {'1.5 Tesla MRI', '3 Tesla MRI'}
    - FSVERSION: {'Cross-Sectional FreeSurfer (FreeSurfer Version 4.3)', 'Cross-Sectional FreeSurfer (5.1)', 'Cross-Sectional FreeSurfer (6.0)'}



In [25]:
# turn outcomes to integer labels
mapping = {'CN': 1, 'MCI': 2, 'Dementia': 3}
df_3to1["DX"] = df_3to1["DX"].map(mapping)
df_3to1["VISCODE"].unique() # check labels
df_3to1["VISCODE"] = df_3to1["VISCODE"].str[1:]
df_3to1["VISCODE"] = df_3to1["VISCODE"].replace('l', '0')
df_3to1["VISCODE"] = df_3to1["VISCODE"].astype(pd.Int64Dtype())

# turn other strings to integer:
dtypes = df_3to1.apply(lambda x:x.dtype)
cate = df_3to1.columns[dtypes=="object"].to_list()
cate = [v for v in cate if v not in ["ABETA", "TAU", "PTAU"]] 
# > don't clean ABETA, TAU, PTAU
lbls = df_3to1.loc[:,cate].apply(lambda x:pd.factorize(x)[1].to_list())
df_3to1.loc[:,cate] = df_3to1.loc[:,cate].apply(lambda x:pd.factorize(x)[0])
df_3to1.loc[:,cate]=df_3to1.loc[:,cate].replace(-1,np.nan) 
# > turn -1 to NaN because default turn NaN to -1
for (col,val) in zip(cate,lbls.to_list()):
    print(f"{col}: {val}")

ORIGPROT: ['ADNI1', 'ADNI2', 'ADNIGO', 'ADNI3']
PTGENDER: ['Male', 'Female']
PTETHCAT: ['Not Hisp/Latino', 'Hisp/Latino']
PTRACCAT: ['White', 'More than one', 'Black', 'Asian', 'Am Indian/Alaskan', 'Hawaiian/Other PI']
PTMARRY: ['Married', 'Divorced', 'Widowed', 'Never married']
FLDSTRENG: ['1.5 Tesla MRI', '3 Tesla MRI']
FSVERSION: ['Cross-Sectional FreeSurfer (FreeSurfer Version 4.3)', 'Cross-Sectional FreeSurfer (5.1)', 'Cross-Sectional FreeSurfer (6.0)']


/var/folders/bm/s0j8htzs7tn4d1t__749xglm0000gn/T/ipykernel_52908/1653795465.py:15: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_3to1.loc[:,cate] = df_3to1.loc[:,cate].apply(lambda x:pd.factorize(x)[0])


In [26]:
d_overview(df_3to1)

Dimension: (11364, 91)


,dtype,NaN_precentage,unique
PIB,float64,0.980465,"[nan, 2.3575, 2.1275, 2.0725, 2.19, 2.27, 2.35..."
FBB,float64,0.955209,"[nan, 1.0714, 1.0372, 0.9902, 0.9959, 0.9602, ..."
TAU_UPENN,float64,0.811246,"[nan, 61.4, 76.5, 41.8, 49.4, 116.0, 120.0, 12..."
PTAU_UPENN,float64,0.809222,"[nan, 20.4, 21.1, 12.9, 13.5, 67.2, 40.5, 28.5..."
ABETA_UPENN,float64,0.809134,"[nan, 131.0, 137.0, 256.0, 246.0, 116.0, 136.0..."
PTAU,object,0.791799,"[nan, 22.83, 24.18, 13.29, 13.3, 33.43, 34.04,..."
TAU,object,0.791711,"[nan, 239.7, 251.7, 153.1, 159.7, 337.0, 334.1..."
ABETA,object,0.791711,"[nan, 741.5, 601.4, 1501.0, 1176.0, 547.3, 472..."
AV45,float64,0.731345,"[nan, 1.3496, 0.9804, 0.9851, 0.994, 0.9416, 0..."
OTHER,float64,0.69113,"[nan, 0.0, 1.0]"


In [27]:
df_3to1.to_csv('../clean_data/df_3to1.csv', index=False)